In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import ee
import multiprocessing
import geemap
import geetools
import datetime

In [2]:
ee.Initialize(project='ee-jonstar')
#ee.Initialize(project='ee-jonstar', opt_url='https://earthengine-highvolume.googleapis.com')

## CONUS Geometry

In [106]:
"""
Points of interest for each urban area.
KEY:
First and second points = Landsat
Third point = Sentinel
If the second point is zero, then there is only one Landsat image needed to cover that city and its surrounding area
The third point is defined near the city center. A circular region of 200 km is created around the third point
to filter Sentinel-1 images, ensuring all of the necessary images will be present.
"""
city_points = {
    'DMV':[ee.Geometry.Point(-76.6122, 39.2904), 0, ee.Geometry.Point(-76.863, 39.063)],
    'NYC':[ee.Geometry.Point(-73.274, 40.688), 0, ee.Geometry.Point(-73.979, 40.751)],
    'Atlanta':[ee.Geometry.Point(-84.039, 34.155), ee.Geometry.Point(-84.485, 33.641), ee.Geometry.Point(-84.38, 33.748)],
    'Miami':[ee.Geometry.Point(-80.267, 25.818), 0, ee.Geometry.Point(-80.267, 25.818)],
    'Chicago':[ee.Geometry.Point(-88.037, 41.826), 0, ee.Geometry.Point(-87.74, 41.78)],
    'Phoenix':[ee.Geometry.Point(-112.02, 33.46), 0, ee.Geometry.Point(-112.064, 33.487)],
    'Denver':[ee.Geometry.Point(-104.72, 39.44), ee.Geometry.Point(-104.47, 39.99), ee.Geometry.Point(-104.985, 39.729)],
    'Seattle':[ee.Geometry.Point(-121.952, 47.546), 0, ee.Geometry.Point(-122.322, 47.59)],
    'San_Francisco':[ee.Geometry.Point(-122.179, 37.745), 0, ee.Geometry.Point(-122.365, 37.745)],
    'Los_Angeles':[ee.Geometry.Point(-118.365, 34.279), ee.Geometry.Point(-118.744, 33.383), ee.Geometry.Point(-122.365, 37.745)],
    'Toronto':[ee.Geometry.Point(-80.273, 43.289), ee.Geometry.Point(-79.609, 44.187), ee.Geometry.Point(-79.46, 43.616)],
    'Mexico_City':[ee.Geometry.Point(-99.101, 19.012), ee.Geometry.Point(-99.187, 20.084), ee.Geometry.Point(-99.129, 19.422)],
    'Las_Vegas':[ee.Geometry.Point(-115.162, 36.154), 0, ee.Geometry.Point(-115.161, 36.154)],
    'Salt_Lake_City':[ee.Geometry.Point(-111.896, 40.752), ee.Geometry.Point(-111.57, 41.37), ee.Geometry.Point(-111.896, 40.752)],
    'Dallas':[ee.Geometry.Point(-96.858, 32.789), 0, ee.Geometry.Point(-96.968, 32.794)],
    'Houston':[ee.Geometry.Point(-94.816, 30.034), ee.Geometry.Point(-95.356, 29.399), ee.Geometry.Point(-95.376, 29.758)],
    'New_Orleans':[ee.Geometry.Point(-90.102, 29.947), 0, ee.Geometry.Point(-90.102, 29.947)],
    'St_Louis':[ee.Geometry.Point(-90.505, 38.644), 0, ee.Geometry.Point(-90.205, 38.624)],
    'Minneapolis':[ee.Geometry.Point(-93.248, 44.969), ee.Geometry.Point(-92.998, 45.555), ee.Geometry.Point(-93.248, 44.969)],
    'Jacksonville':[ee.Geometry.Point(-81.357, 30.292), 0, ee.Geometry.Point(-81.651, 30.327)],
    'Charlotte':[ee.Geometry.Point(-80.797, 35.028), ee.Geometry.Point(-80.573, 35.63), ee.Geometry.Point(-80.842, 35.218)],
    'Philadelphia':[ee.Geometry.Point(-75.133, 39.938), 0, ee.Geometry.Point(-75.154, 39.939)],
    'San_Diego':[ee.Geometry.Point(-117.088, 32.647), 0, ee.Geometry.Point(-117.093, 32.578)],
    'San_Juan':[ee.Geometry.Point(-66.311, 18.421), 0, ee.Geometry.Point(-66.066, 18.407)],
    'Montreal':[ee.Geometry.Point(-73.353, 45.148), ee.Geometry.Point(-73.082, 45.629), ee.Geometry.Point(-73.59, 45.52)],
    'Guadalajara':[ee.Geometry.Point(-103.35, 20.663), 0, ee.Geometry.Point(-103.35, 20.663)],
    'Monterrey':[ee.Geometry.Point(-100.17, 25.897), 0, ee.Geometry.Point(-100.318, 25.711)],
    'Cancun':[ee.Geometry.Point(-86.592, 21.402), ee.Geometry.Point(-86.725, 20.709), ee.Geometry.Point(-86.85, 21.161)],
    'Billings':[ee.Geometry.Point(-108.5, 45.78), 0, ee.Geometry.Point(-108.504, 45.781)],
    'Guatemala_City':[ee.Geometry.Point(-90.527, 14.623), 0, ee.Geometry.Point(-90.527, 14.623)],
    'San_Jose':[ee.Geometry.Point(-84.082, 9.937), 0, ee.Geometry.Point(-84.082, 9.937)],
    'Havana':[ee.Geometry.Point(-82.36, 23.128), 0, ee.Geometry.Point(-82.36, 23.128)],
    'Santo_Domingo':[ee.Geometry.Point(-69.822, 18.681), 0, ee.Geometry.Point(-69.932, 18.468)],
    'Tegucigalpa':[ee.Geometry.Point(-87.206, 14.171), 0, ee.Geometry.Point(-87.197, 14.057)],
    'Managua':[ee.Geometry.Point(-86.269, 12.017), ee.Geometry.Point(-86.145, 12.349), ee.Geometry.Point(-86.239, 12.097)],
    'Panama_City':[ee.Geometry.Point(-79.522, 8.98), 0, ee.Geometry.Point(-79.522, 8.98)],
    'Bogota':[ee.Geometry.Point(-74.074, 4.697), 0, ee.Geometry.Point(-74.074, 4.697)],
    'Lima':[ee.Geometry.Point(-76.887, -11.835), ee.Geometry.Point(-76.99, -12.383), ee.Geometry.Point(-77.065, -12.066)],
    'Quito':[ee.Geometry.Point(-78.507, -.232), 0, ee.Geometry.Point(-78.507, -0.232)],
    'Santiago':[ee.Geometry.Point(-70.667, -33.46), 0, ee.Geometry.Point(-70.667, -33.46)],
    'Buenos_Aires':[ee.Geometry.Point(-58.543, -34.621), 0, ee.Geometry.Point(-58.458, -34.666)],
    'Sao_Paulo':[ee.Geometry.Point(-46.644, -23.564), 0, ee.Geometry.Point(-46.644, -23.564)],
    'Manaus':[ee.Geometry.Point(-60.037, -3.089), 0, ee.Geometry.Point(-60.037, -3.089)],
    'Punta_Arenas':[ee.Geometry.Point(-71.327, -52.544), ee.Geometry.Point(-70.179, -52.885), ee.Geometry.Point(-70.909, -53.157)],
    'La_Paz':[ee.Geometry.Point(-67.885, -17.004), ee.Geometry.Point(-67.965, -16.125), ee.Geometry.Point(-68.143, -16.522)],
    'Montevideo':[ee.Geometry.Point(-56.175, -34.902), 0, ee.Geometry.Point(-56.175, -34.902)],
    'Brasilia':[ee.Geometry.Point(-47.890, -15.799), 0, ee.Geometry.Point(-47.89, -15.799)],
    'Caracas':[ee.Geometry.Point(-66.898, 10.475), 0, ee.Geometry.Point(-66.907, 10.474)]
}

In [84]:
"""
Export coordinates for each urban area
KEY: [utm zone, utm x, utm y]
"""
export_coords = {
    'DMV':[18, True, 292000, 4372200],
    'NYC':[18, True, 562500, 4537469],
    'Phoenix':[12, True, 350000, 3749000],
    'Miami':[17, True, 503991, 2915332],
    'Chicago':[16, True, 386195, 4680677],
    'Denver':[13, True, 466349, 4442816],
    'Seattle':[10, True, 504628, 5311856],
    'San_Francisco':[10, True, 541523, 4206919],
    'Los_Angeles':[11, True, 352871, 3803787],
    'Atlanta':[16, True, 703143, 3787639],
    'Toronto':[17, True, 566749, 4867878],
    'Mexico_City':[14, True, 426738, 2202781],
    'Las_Vegas':[11, True, 618776, 4038562],
    'Salt_Lake_City':[12, True, 387959, 4539501],
    'Dallas':[14, True, 639754, 3678675],
    'Houston':[15, True, 230679, 3334493],
    'New_Orleans':[15, True, 723933, 3361201],
    'St_Louis':[15, True, 675752, 4320402],
    'Minneapolis':[15, True, 433510, 5031071],
    'Jacksonville':[17, True, 421321, 3375918],
    'Charlotte':[17, True, 463969, 3944648],
    'Philadelphia':[18, True, 445906, 4467022],
    'San_Diego':[11, True, 468102, 3666860],
    'San_Juan':[19, True, 732591, 2098412],
    'Montreal':[18, True, 563054, 5091925],
    'Guadalajara':[13, True, 621576, 2326245],
    'Monterrey':[14, True, 318956, 2885247],
    'Cancun':[16, True, 491502, 2369076],
    'Billings':[12, True, 655301, 5109344],
    'Guatemala_City':[15, True, 711929, 1659823],
    'San_Jose':[16, True, 772982, 1148166],
    'Havana':[17, True, 310663, 2566532],
    'Santo_Domingo':[19, True, 354586, 2108039],
    'Tegucigalpa':[16, True, 401921, 1598242],
    'Managua':[16, True, 524987, 1386278],
    'Panama_City':[17, True, 610881, 1043658],
    'Bogota':[18, True, 546553, 562185],
    'Lima':[18, False, 262348, 8709133],
    'Quito':[17, True, 730387, 29863],
    'Santiago':[19, False, 288729, 6352433],
    'Buenos_Aires':[21, 326030, 6204959],
    'Sao_Paulo':[23, False, 294104, 7446208],
    'Manaus':[20, False, 762402, 9705737],
    'Punta_Arenas':[19, False, 353001, 4162220],
    'La_Paz':[19, False, 548109, 8213756],
    'Montevideo':[21, False, 545458, 6223320],
    'Brasilia':[22, False, 799612, 8288848],
    'Caracas':[19, True, 686519, 1174917]
}

In [85]:
# Pull points to make data for a specific city (look above for options)
city = 'Houston'
points = city_points[city]
city_export = export_coords[city]
if city_export[1]:
    crs_prefix = '326'
else:
    crs_prefix = '327'
utm_proj = f'EPSG:{crs_prefix}{city_export[0]}'
point = ee.Geometry.Point([city_export[2]+30*2999/2,city_export[3]-30*2999/2], utm_proj)
export_region = point.buffer(distance=44990, proj=utm_proj).bounds(proj=utm_proj)
city_export

[15, True, 230679, 3334493]

## Landsat Initialization

In [86]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [87]:
landsat = landsat.filterDate('2022-01-01', '2024-01-01')
landsat_US = landsat.filterBounds(points[0])
if points[1] != 0:
    #pt1_filt = ee.Filter.bounds(points[0])
    #pt2_filt = ee.Filter.bounds(points[1])
    #Landsat_pts_filt = ee.Filter.Or(pt1_filt, pt2_filt)
    #landsat_US = landsat.filter(Landsat_pts_filt)
    landsat_US2 = landsat.filterBounds(points[1])
    print('Combine Landsat')
elif points[0] == 0:
    raise Exception('Points not formatted correctly')

Combine Landsat


In [88]:
landsat_US.first().projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32615',
 'transform': [30, 0, 215385, 0, -30, 3471615]}

In [89]:
# Preprocessing function
def process_multiple_Landsat(image):
    ######################################
    # Combines Landsat images matched by time into one image
    
    match = ee.Image(image.get('matched_img'))
    median_image = ee.ImageCollection([image, match]).median()

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(image.get('system:time_start'))       
        return f.set('system:time_start', Landsat_time)

    # Add timestamp back in
    median_image = get_Landsat_time(median_image)
    
    return median_image

In [90]:
# Join sequential Landsat images
if points[1] != 0:
    best_join = ee.Join.saveBest(matchKey='matched_img', measureKey='time_diff', outer=False)
    # Difference value is equal to 1 minute (for sequential Landsat images)
    timeFiltLandsat = ee.Filter.maxDifference(difference=60e3, leftField='system:time_start', rightField='system:time_start')
    Landsat_near = ee.ImageCollection(best_join.apply(landsat_US, landsat_US2, timeFiltLandsat))
    landsat_US = Landsat_near.map(process_multiple_Landsat)
    print('Landsat join')

Landsat join


In [91]:
print(landsat_US.size().getInfo())

36


## Sentinel-1 SAR Initialization

In [92]:
sentinel = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate('2022-01-01', '2024-01-01').filterBounds(export_region)

In [93]:
slist = sentinel.toList(10)

In [94]:
ee.Image(slist.get(1)).select('angle').projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32615',
 'transform': [-12350.692933820188,
  -4129.204427858873,
  354964.7989228259,
  2545.7842329894193,
  -19998.554017739836,
  3390591.7249016208]}

In [95]:
print(sentinel.size().getInfo())

287


## Filtering images by time

In [96]:
# Join Landsat and Sentinel images
all_join_LS = ee.Join.saveAll(matchesKey='matched_img', measureKey='time_diff', outer=True)
# Difference value is equal to 6 days (Full Sentinel-1 orbital period) in milliseconds
timeFiltLandsatSentinel = ee.Filter.maxDifference(difference=5184e5, leftField='system:time_start', rightField='system:time_start')
LandsatSentinel_near = ee.ImageCollection(all_join_LS.apply(landsat_US, sentinel, timeFiltLandsatSentinel))

In [97]:
LandsatSentinel_near.size().getInfo()

36

## Helper Functions

In [18]:
# Applies scale and offset factors for Landsat bands
def scale_and_offset_Landsat(image):
    reflect_weight = 0.0000275
    reflect_bias = -0.2
    reflect_image = image.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(reflect_weight).add(reflect_bias)

    LST_weight = 0.00341802
    LST_bias = 149
    LST_image = image.select('ST_B10').multiply(LST_weight).add(LST_bias)

    return reflect_image.addBands(srcImg=LST_image, names=['ST_B10'])

In [19]:
# Applies scale and offset factors for GOES imagery
# All scale and offset values were taken straight from GEE to save compute
def scale_and_offset_GOES(image):
    #reflectances = image.select(['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06']).multiply(0.0002442)
    weight = 0.039316241
    bias = 173.14999
    brightness_temps = image.select(['CMI_C13', 'CMI_C14', 'CMI_C15', 'CMI_C16']).multiply(weight).add(bias)
    
    #return reflectances.addBands(srcImg=brightness_temps, names=['CMI_C14', 'CMI_C15'])
    return brightness_temps

In [20]:
# Preprocessing function
def process_GOES(image):
    ######################################
    # GOES portion

    # Scaling and offset
    #image2 = ee.Image(feature.get('matched_img'))
    GOES_image = scale_and_offset_GOES(image)

    ######################################
    # Timestamp portion
    
    # Function to get time from GOES image\    
    def get_GOES_time(f):
        GOES_time = ee.Number(image.get('system:time_start'))       
        return f.set('timestamp', GOES_time)

    # Add timestamp back in
    #landsat_image = get_GOES_time(landsat_image)
    GOES_image = get_GOES_time(GOES_image)

    ##########################################
    
    #image = landsat_image.addBands(srcImg=GOES_image, names=['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05',
    #                                                         'CMI_C06', 'CMI_C14', 'CMI_C15']).cast({'QA_PIXEL':'double'})
    return GOES_image

In [19]:
# Function to add missing bands
def add_missing_bands(image):
    # Define the complete set of bands you want to work with
    all_bands = ['VV', 'VH', 'HH', 'HV', 'angle']
    constants = ee.Dictionary({'VV':2, 'VH':2, 'HH':2, 'HV':2, 'angle':-999})
    
    # Get the bands present in the image
    existing_bands = image.bandNames()
    # Determine the missing bands
    missing_bands = ee.List(all_bands).removeAll(existing_bands)
    # Add missing bands with a default value of 2
    filled_image = missing_bands.iterate(
        lambda band, img: ee.Image(img).addBands(ee.Image.random().add(ee.Image.constant(constants.get(band))).rename([ee.String(band)])),
        image
    )
    # Ensure the image has all bands in the correct order
    return ee.Image(filled_image).select(all_bands)

def process_multiple_sentinel(feature):
    Sentinel_col = ee.ImageCollection(ee.List(feature.get('matched_img')))
    
    # Define the complete set of bands you want to work with
    all_bands = ['VV', 'VH', 'HH', 'HV', 'angle']

    # Want most recent images last in collection for mosaic
    sorted_collection = Sentinel_col.sort('time_diff', ascending=False).map(add_missing_bands)
    Sentinel_image = sorted_collection.mosaic()
    
    return Sentinel_image.cast({'angle':'double'})

In [20]:
# Preprocessing function
def process_Landsat_Sentinel(feature):
    ######################################
    # Sentinel portion
               
    # Three-case if:
    # 1st: More than one image match -> combine into one image
    # Second: No image match -> return blank image
    # Third: One image match -> return singular image
    sentinel_image = ee.Algorithms.If(ee.List(feature.get('matched_img')).length().gt(ee.Number(1)),
                     process_multiple_sentinel(feature),
                     ee.Algorithms.If(ee.List(feature.get('matched_img')).length().lt(ee.Number(1)),
                                      add_missing_bands(ee.Image()),
                                      add_missing_bands(ee.Image(ee.List(feature.get('matched_img')).get(0)).cast({'angle':'double'})))
                                     )
    
    #####################################
    # Landsat portion
    
    # Scaling and offset
    feature = ee.Image(feature)
    #image1 = ee.Image(feature.get('primary'))
    landsat = scale_and_offset_Landsat(feature)
    #landsat_image = scale_and_offset_Landsat(feature)
    # Add cloud mask back in
    landsat_image = landsat.addBands(srcImg=feature, names=['QA_PIXEL'])

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(feature.get('system:time_start'))       
        return f.set('timestamp', Landsat_time)

    # Add timestamp back in
    landsat_image = get_Landsat_time(landsat_image)

    ##########################################
    image = landsat_image.addBands(srcImg=sentinel_image).cast({'QA_PIXEL':'double'})
    #image = landsat_image.cast({'QA_PIXEL':'double'})
    
    return image

## Produce GOES geotiffs and their respective times

In [21]:
def times_to_features(num):
    return ee.Feature(None, {'value': num})

In [14]:
processed = GOES.map(process_GOES)

In [53]:
first = processed.first()

In [41]:
#times = processed.aggregate_array('timestamp')
times = GOES.aggregate_array('system:time_start')
features = ee.FeatureCollection(times.map(times_to_features))

NameError: name 'GOES' is not defined

In [13]:
task = ee.batch.Export.table.toDrive(
    collection=features,
    description='GOES_West_times',
    folder='',
    fileFormat='CSV',
)
task.start()

## Produce Landsat/Sentinel geotiffs and their respective times

In [98]:
processed2 = LandsatSentinel_near.map(process_Landsat_Sentinel)

In [301]:
plist = processed2.toList(30)

In [302]:
ee.Image(plist.get(0)).getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7651, 7781],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 527085, 0, -30, 4582215]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7651, 7781],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 527085, 0, -30, 4582215]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7651, 7781],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 527085, 0, -30, 4582215]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7651, 7781],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 527085, 0, -30, 4582215]},
  {'id': 'SR_B6',
   'data_type': {'type'

In [61]:
processed2.size().getInfo()

36

In [193]:
times2 = processed2.aggregate_array('timestamp')
features2 = ee.FeatureCollection(times2.map(times_to_features))

In [194]:
task = ee.batch.Export.table.toDrive(
    collection=features2,
    description=f'Landsat_times_{city}',
    #folder=f'Landsat_times',
    fileFormat='CSV',
)
task.start()

## Export stuff

In [103]:
# Must first download the times files created above in the previous two sections
ls_times = pd.read_csv(f'/Users/jonstar/Documents/heat_data/Landsat_times/Landsat_times_{city}.csv').value

In [104]:
datetime.datetime.fromtimestamp(ls_times[5]/1000, datetime.UTC).strftime('%Y%m%d%H%M')

'202204201650'

In [119]:
# Export Landsat/Sentinel images
num = processed2.size().getInfo()
#num = 4
img_list = processed2.toList(num)

for i in list(range(num-4)):
    i = i+4
    img = ee.Image(img_list.get(i))
    time_str = toTimezone(datetime.fromtimestamp(ls_times[i]/1000), pytz.utc).strftime('%Y%m%d%H%M')
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{time_str}', fileFormat='GeoTIFF', folder='Landsat_SAR_DMV_new',
                dimensions='3000x3000', crs=f'EPSG:{crs_prefix}{city_export[0]}', crsTransform=[30, 0, city_export[1], 0, -30, city_export[2]],
        formatOptions={'cloudOptimized':True})
    task.start()

In [114]:
# Export Landsat images for export to GEE asset for visualizing
num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toAsset(
        img, description=f'Landsat_Sentinel_image_{i}', assetId=f'projects/ee-jonstar/assets/{city}_LS_test',
                dimensions='3000x3000', crs=f'EPSG:{crs_prefix}{city_export[0]}', crsTransform=[30, 0, city_export[1], 0, -30, city_export[2]])
    task.start()

In [105]:
# Export Landsat/Sentinel images new way
num = processed2.size().getInfo()
#num = 1
img_list = processed2.toList(num)
point = ee.Geometry.Point([city_export[2]+30*2999/2,city_export[3]-30*2999/2], f'EPSG:{crs_prefix}{city_export[0]}')

for i in list(range(num)):
#for i in list(range(30,45)):
    img = ee.Image(img_list.get(i))
    time_str = datetime.datetime.fromtimestamp(ls_times[i]/1000, datetime.UTC).strftime('%Y%m%d%H%M')
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{city}_{time_str}', fileFormat='GeoTIFF', #folder=f'Landsat_SAR_{city}',
        region = export_region, crs=utm_proj, scale=30,
        formatOptions={'cloudOptimized':True})
    task.start()

In [101]:
# Export Landsat/Sentinel images new way as assets
#num = processed2.size().getInfo()
num = 1
img_list = processed2.toList(num)
point = ee.Geometry.Point([city_export[2]+30*2999/2,city_export[3]-30*2999/2], f'EPSG:{crs_prefix}{city_export[0]}')

for i in list(range(num)):
#if i in [19]:
    img = ee.Image(img_list.get(i))
    time_str = datetime.datetime.fromtimestamp(ls_times[i]/1000, datetime.UTC).strftime('%Y%m%d%H%M')
    
    task = ee.batch.Export.image.toAsset(
        img, description=f'Landsat_Sentinel_image_{time_str}', assetId=f'projects/ee-jonstar/assets/{city}_LS_test',
        region = export_region,
        crs='EPSG:4326', scale=30)
    task.start()